In [13]:
import os
import time
from typing import List, Optional

import pandas as pd
import rich
import wandb
from rich import print
from rich.console import Console
from rich.table import Table
from tqdm.auto import tqdm

def display_pretty(df):
    # visualize using tabulate for ipython noteboooks
    from IPython.display import display

    # Assuming df is your DataFrame
    with pd.option_context("display.max_rows", None, "display.max_columns", None):
        display(
            df
        )



def login(project: Optional[str] = None, token: Optional[str] = None):
    if project is None:
        project = os.getenv("WANDB_PROJECT")
    if token is None:
        token = os.getenv("WANDB_API_KEY")
    # Set your wandb API key
    wandb.login(key=token)


def fetch_runs(
    project_list,
    exp_name_list: Optional[List[str]] = None,
    exp_term_list: Optional[List[str]] = None,
    total_runs_limit: Optional[int] = None,
):
    # Fetch runs from wandb
    api = wandb.Api()
    runs = (
        api.runs(path=project_list)
        if isinstance(project_list, str)
        else [api.runs(path=p) for p in project_list]
    )
    # runs = [run for run_list in runs for run in run_list]

    exp_name_to_summary_dict = {}
    exp_name_to_state_dict = {}
    idx = 0
    for run in tqdm(runs[0]):
        if "exp_name" not in run.config:
            continue
        exp_name = run.config["exp_name"].lower()

        if exp_name_list is not None and exp_name not in exp_name_list:
            continue

        if exp_term_list is not None:
            if not any(term in exp_name for term in exp_term_list):
                continue

        if exp_name not in exp_name_to_summary_dict:
            exp_name_to_summary_dict[exp_name] = [run.summaryMetrics]
            exp_name_to_state_dict[exp_name] = [run.state]
        else:
            exp_name_to_summary_dict[exp_name].append(run.summaryMetrics)
            exp_name_to_state_dict[exp_name].append(run.state)
        if total_runs_limit is not None and idx >= total_runs_limit:
            break
        idx += 1
    # sort the dicts by exp_name
    exp_name_to_summary_dict = dict(sorted(exp_name_to_summary_dict.items()))
    exp_name_to_state_dict = dict(sorted(exp_name_to_state_dict.items()))
    return exp_name_to_summary_dict, exp_name_to_state_dict


def fetch_run_status(exp_name_to_summary_dict, exp_name_to_state_dict):
    exp_data = {}
    exp_to_command = {
        key.lower(): value for key, value in exp_name_to_summary_dict.items()
    }
    if isinstance(exp_name_to_summary_dict, dict):
        experiments = list(exp_name_to_summary_dict.keys())
    experiments = [exp.lower() for exp in experiments]
    # Iterate through the given experiments
    with tqdm(total=len(experiments), desc="Checking experiments") as pbar:
        for exp_name in experiments:
            # Check if the experiment exists in wandb and if it has completed the testing stage
            if exp_name in exp_name_to_summary_dict:
                keys = [
                    k
                    for summary_keys in exp_name_to_summary_dict[exp_name]
                    for k in summary_keys.keys()
                ]

                testing_completed = any("testing/ensemble" in k for k in keys)
                model_compiled = any("model/num_parameters" in k for k in keys)

                currently_running = any(
                    "running" == state.lower()
                    for state in exp_name_to_state_dict[exp_name]
                )
                if "global_step" in keys:
                    current_iter = max(
                        [
                            summary_stats["global_step"]
                            for summary_stats in exp_name_to_summary_dict[
                                exp_name
                            ]
                            if "global_step" in summary_stats.keys()
                        ]
                    )
                else:
                    current_iter = 0

            else:
                testing_completed = False
                currently_running = False
                current_iter = 0

            # Append the data to the list
            exp_data[exp_name] = {
                "testing_completed": testing_completed,
                "currently_running": currently_running,
                "current_iter": current_iter,
                "model_compiled": model_compiled,
                "command": exp_to_command[exp_name],
            }
            pbar.update(1)
    return exp_data


def pretty_print_runs(exp_data):

    # Create a pandas DataFrame
    df = pd.DataFrame(
        exp_data
    ).T  # Transpose the DataFrame so that each experiment is a row

    # Create a console for rich print
    console = Console()

    # Create a table
    table = Table(show_header=True, header_style="bold magenta", style="dim")
    table.add_column("idx", justify="right")
    table.add_column("Experiment Name", width=50)
    table.add_column("Currently Running", justify="right")
    table.add_column("Testing Completed", justify="right")
    table.add_column("Current Iteration", justify="right")
    table.add_column("Model Compiled", justify="right")

    # Add rows to the table
    for idx, (exp_name, row) in enumerate(df.iterrows()):
        table.add_row(
            str(idx),
            exp_name,
            str(row["currently_running"]),
            str(row["testing_completed"]),
            str(row["current_iter"]),
            str(row["model_compiled"]),
        )

    # Print the table
    console.print(table)

model_names = ["ar-vit-b16", "effformer-s0", "bart", "bert", "clip-b16", "whisper", "svit-b16", "siglip-p16", "rnx50-32x4a1", "mpnet", "laion-b16", "flex-b-1200ep", "effv2-rw-s", "dino-b16", "deit3-b16", "convnextv2-base"]


In [34]:
project = "machinelearningbrewery/gate-0-9-1"
token = None
model_names = list(set(model_names))

exp_name_to_summary_dict, exp_name_to_state_dict = fetch_runs(
                project_list=[project],
                exp_name_list=None,
                exp_term_list=["9032024"],
                total_runs_limit=None,
            )
exp_data = fetch_run_status(
exp_name_to_summary_dict, exp_name_to_state_dict
)
            
pretty_print_runs(exp_data)

# store the dicts in json
import json

with open("exp_data.json", "w") as f:
    json.dump(exp_data, f)

# store the dicts in json
with open("exp_name_to_summary_dict.json", "w") as f:
    json.dump(exp_name_to_summary_dict, f)

# store the dicts in json
with open("exp_name_to_state_dict.json", "w") as f:
    json.dump(exp_name_to_state_dict, f)

  0%|          | 0/4265 [00:00<?, ?it/s]

Checking experiments:   0%|          | 0/419 [00:00<?, ?it/s]

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃     ┃                                                    ┃   Currently ┃     Testing ┃    Current ┃       Model ┃
┃ idx ┃ Experiment Name                                    ┃     Running ┃   Completed ┃  Iteration ┃    Compiled ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│   0 │ debug-acdc-ar-vit-b16-512-9032024                  │       False │        True │      10001 │        True │
│   1 │ debug-acdc-bart-9032024                            │       False │        True │      10002 │        True │
│   2 │ debug-acdc-bert-9032024                            │       False │       False │          0 │        True │
│   3 │ debug-acdc-clip-b16-512-9032024                    │       False │        True │      10001 │        True │
│   4 │ debug-acdc-convnextv2-base-9032024                 │       False │       False │          3 │        True │
│   5 │ debug-acdc-deit3-b16-512-9032024                   │       False │        True │      10002 │        True │
│   6 │ debug-acdc-dino-b16-512-9032024                    │       False │        True │      10001 │        True │
│   7 │ debug-acdc-effv2-rw-s-9032024                      │       False │        True │      10002 │        True │
│   8 │ debug-acdc-flex-b-1200ep-9032024                   │       False │        True │      10002 │        True │
│   9 │ debug-acdc-laion-b16-512-9032024                   │       False │        True │      10001 │        True │
│  10 │ debug-acdc-mpnet-9032024                           │       False │       False │          0 │        True │
│  11 │ debug-acdc-rnx50-32x4a1-9032024                    │       False │        True │      10001 │        True │
│  12 │ debug-acdc-siglip-p16-512-9032024                  │       False │        True │      10001 │        True │
│  13 │ debug-acdc-svit-b16-512-9032024                    │       False │        True │      10001 │        True │
│  14 │ debug-acdc-whisper-9032024                         │       False │        True │      10001 │        True │
│  15 │ debug-ade20k-ar-vit-b16-1024-hpo-0-9032024         │       False │        True │      10001 │        True │
│  16 │ debug-ade20k-bart-hpo-0-9032024                    │       False │        True │      10001 │        True │
│  17 │ debug-ade20k-bert-hpo-0-9032024                    │       False │       False │          0 │        True │
│  18 │ debug-ade20k-clip-b16-1024-hpo-0-9032024           │       False │        True │      10001 │        True │
│  19 │ debug-ade20k-convnextv2-base-hpo-0-9032024         │       False │       False │          0 │        True │
│  20 │ debug-ade20k-deit3-b16-1024-hpo-0-9032024          │       False │        True │      10001 │        True │
│  21 │ debug-ade20k-dino-b16-1024-hpo-0-9032024           │       False │        True │      10001 │        True │
│  22 │ debug-ade20k-effv2-rw-s-hpo-0-9032024              │       False │        True │      10001 │        True │
│  23 │ debug-ade20k-flex-b-1200ep-hpo-0-9032024           │       False │        True │      10001 │        True │
│  24 │ debug-ade20k-laion-b16-1024-hpo-0-9032024          │       False │        True │      10001 │        True │
│  25 │ debug-ade20k-mpnet-hpo-0-9032024                   │       False │       False │          0 │        True │
│  26 │ debug-ade20k-rnx50-32x4a1-hpo-0-9032024            │       False │        True │      10001 │        True │
│  27 │ debug-ade20k-siglip-p16-1024-hpo-0-9032024         │       False │        True │      10001 │        True │
│  28 │ debug-ade20k-svit-b16-1024-hpo-0-9032024           │       False │        True │      10001 │        True │
│  29 │ debug-ade20k-whisper-hpo-0-9032024                 │       False │        True │      10001 │        True │
│  30 │ debug-aircraft-fs-classification-ar-vit-b16-224-

In [35]:
import json
# load the json
with open("exp_data.json", "r") as f:
    exp_data = json.load(f)

# load the json
with open("exp_name_to_summary_dict.json", "r") as f:
    exp_name_to_summary_dict = json.load(f)

# load the json
with open("exp_name_to_state_dict.json", "r") as f:
    exp_name_to_state_dict = json.load(f)

In [39]:
keys = []
for model_name, metrics in exp_name_to_summary_dict.items():
    cur_keys = [k for item in metrics for k in item.keys() if "testing" in k and "std" not in k and "episode" not in k and "background" not in k and "complete" not in k and "ensemble_3" in k and "similarities" not in k]
    
    keys.extend(cur_keys)

keys = sorted(set(keys))
# print(keys)

results = {}

for model_name in exp_name_to_summary_dict.keys():
    results[model_name] = {}
    for key in keys:
        for item in exp_name_to_summary_dict[model_name]:
            if key in item:
                dataset_name = model_name.split("-")[1] if "math" not in model_name else model_name.split("-")[1] + "-" + model_name.split("-")[2]
                results[model_name][f"{dataset_name}.{key}"] = item[key]
# delete all keys that have no value
for model_name in results.keys():
    for key in list(results[model_name].keys()):
        if results[model_name][key] is None:
            del results[model_name][key]


original_df = pd.DataFrame(results).T
# display_pretty(original_df)
df = original_df.copy()


In [40]:
# label the first column as exp_name
df.index.name = "exp_name"
df["exp_name"] = df.index
df["experiment_series"] = df.index.str.split("-").str[0]
df["dataset_name"] = df.index.str.split("-").str[1]
# break the exp_name column by - and take the first part to be experiment series, and then check if the second part matches one of the model names in the list, and assign that as the model name
# check the exp_name column and if one of the model names matches part of the exp_name, assign it to the model_name column
# for item in df["exp_name"]:
#     for model_name in model_names:
#         print(model_name, item)
#         if model_name in item:
#             print(f"Model name: {model_name}, exp_name: {item}")
#             continue
#     print(f"Model name: None, exp_name: {item}")
df["model_name"] = df["exp_name"].apply(lambda x: next((model_name for model_name in model_names if model_name in x), None))
# set model_name to be in the second column
df = df.set_index("dataset_name", append=True)
df = df.set_index("model_name", append=True)
df = df.set_index("experiment_series", append=True)
df = df.groupby('model_name').first().reset_index()
# if a column has only None and NaNs it should be removed
df = df.dropna(axis=0, how='all')



# remove dataset_name column
# df = df.drop(columns=['dataset_name'])
# unify the rows by model_name


display_pretty(df)

,model_name,acdc.testing/ensemble_3--/dice_loss-epoch-mean,acdc.testing/ensemble_3--/loss-epoch-mean,acdc.testing/ensemble_3-/dice_loss-epoch-mean,acdc.testing/ensemble_3-/loss-epoch-mean,acdc.testing/ensemble_3-/mIoU,acdc.testing/ensemble_3-/mean_accuracy,acdc.testing/ensemble_3-/overall_accuracy,ade20k.testing/ensemble_3--/ce_loss-epoch-mean,ade20k.testing/ensemble_3--/dice_loss-epoch-mean,ade20k.testing/ensemble_3--/focal_loss-epoch-mean,ade20k.testing/ensemble_3--/loss-epoch-mean,ade20k.testing/ensemble_3-/ce_loss-epoch-mean,ade20k.testing/ensemble_3-/dice_loss-epoch-mean,ade20k.testing/ensemble_3-/focal_loss-epoch-mean,ade20k.testing/ensemble_3-/loss-epoch-mean,ade20k.testing/ensemble_3-/mIoU,ade20k.testing/ensemble_3-/mean_accuracy,ade20k.testing/ensemble_3-/overall_accuracy,aircraft.testing/ensemble_3-/accuracy_top_1-epoch-mean,aircraft.testing/ensemble_3-/logits-epoch-mean,aircraft.testing/ensemble_3-/loss-epoch-mean,chexpert.testing/ensemble_3/0-aps,chexpert.testing/ensemble_3/0-auc,chexpert.testing/ensemble_3/0-bs,chexpert.testing/ensemble_3/1-aps,chexpert.testing/ensemble_3/1-auc,chexpert.testing/ensemble_3/1-bs,chexpert.testing/ensemble_3/2-aps,chexpert.testing/ensemble_3/2-auc,chexpert.testing/ensemble_3/2-bs,chexpert.testing/ensemble_3/3-aps,chexpert.testing/ensemble_3/3-auc,chexpert.testing/ensemble_3/3-bs,chexpert.testing/ensemble_3/4-aps,chexpert.testing/ensemble_3/4-auc,chexpert.testing/ensemble_3/4-bs,chexpert.testing/ensemble_3/aps-macro,chexpert.testing/ensemble_3/auc-macro,chexpert.testing/ensemble_3/bs-macro,chexpert.testing/ensemble_3/ensemble_3-global_step,chexpert.testing/ensemble_3/loss,cifar100.testing/ensemble_3-/accuracy_top_1-epoch-mean,cifar100.testing/ensemble_3-/accuracy_top_5-epoch-mean,cifar100.testing/ensemble_3-/logits-epoch-mean,cifar100.testing/ensemble_3-/loss-epoch-mean,clevr.testing/ensemble_3-/accuracy_top_1-epoch-mean,clevr.testing/ensemble_3-/accuracy_top_1_colour-epoch-mean,clevr.testing/ensemble_3-/accuracy_top_1_count-epoch-mean,clevr.testing/ensemble_3-/accuracy_top_1_material-epoch-mean,clevr.testing/ensemble_3-/accuracy_top_1_shape-epoch-mean,clevr.testing/ensemble_3-/accuracy_top_1_size-epoch-mean,clevr.testing/ensemble_3-/accuracy_top_1_yes_no-epoch-mean,clevr.testing/ensemble_3-/logits_colour-epoch-mean,clevr.testing/ensemble_3-/logits_count-epoch-mean,clevr.testing/ensemble_3-/logits_material-epoch-mean,clevr.testing/ensemble_3-/logits_shape-epoch-mean,clevr.testing/ensemble_3-/logits_size-epoch-mean,clevr.testing/ensemble_3-/logits_yes_no-epoch-mean,clevr.testing/ensemble_3-/loss-epoch-mean,clevr.testing/ensemble_3-/loss_colour-epoch-mean,clevr.testing/ensemble_3-/loss_count-epoch-mean,clevr.testing/ensemble_3-/loss_material-epoch-mean,clevr.testing/ensemble_3-/loss_shape-epoch-mean,clevr.testing/ensemble_3-/loss_size-epoch-mean,clevr.testing/ensemble_3-/loss_yes_no-epoch-mean,clevr-math.testing/ensemble_3-/accuracy_top_1-epoch-mean,clevr-math.testing/ensemble_3-/accuracy_top_5-epoch-mean,clevr-math.testing/ensemble_3-/logits-epoch-mean,clevr-math.testing/ensemble_3-/loss-epoch-mean,coco.testing/ensemble_3--/ce_loss-epoch-mean,coco.testing/ensemble_3--/dice_loss-epoch-mean,coco.testing/ensemble_3--/focal_loss-epoch-mean,coco.testing/ensemble_3--/loss-epoch-mean,coco.testing/ensemble_3-/ce_loss-epoch-mean,coco.testing/ensemble_3-/dice_loss-epoch-mean,coco.testing/ensemble_3-/focal_loss-epoch-mean,coco.testing/ensemble_3-/loss-epoch-mean,coco.testing/ensemble_3-/mIoU,coco.testing/ensemble_3-/mean_accuracy,coco.testing/ensemble_3-/overall_accuracy,cubirds.testing/ensemble_3-/accuracy_top_1-epoch-mean,cubirds.testing/ensemble_3-/logits-epoch-mean,cubirds.testing/ensemble_3-/loss-epoch-mean,diabetic.testing/ensemble_3/0-aps,diabetic.testing/ensemble_3/0-auc,diabetic.testing/ensemble_3/0-bs,diabetic.testing/ensemble_3/1-aps,diabetic.testing/ensemble_3/1-auc,diabetic.testing/ensemble_3/1-bs,diabetic.testing/ensemble_3/2-aps,diabetic.testing/ensemble_3/2-auc,diabetic.testing/ens

In [49]:
df = df.dropna(axis=1, how='all')

cleaned_df = df.drop(columns=[col for col in df.columns if df[col].isnull().sum() >= 14])
# remove columns with "logits" in the name
cleaned_df = cleaned_df[[col for col in cleaned_df.columns if "logits" not in col]]
cleaned_df = cleaned_df[[col for col in cleaned_df.columns if "ensemble_3--" not in col]]
cleaned_df = cleaned_df[[col for col in cleaned_df.columns if "global_step" not in col]]


# remove dataset_name column
# df = df.drop(columns=['dataset_name'])
# unify the rows by model_name

# remove columns where all values are 1.0
cleaned_df = cleaned_df[[col for col in cleaned_df.columns if cleaned_df[col].nunique() > 1]]
# replace testing/ensemble_3-/ in column names with .
cleaned_df.columns = cleaned_df.columns.str.replace("testing/ensemble_3-/", "")
cleaned_df.columns = cleaned_df.columns.str.replace("-epoch-mean", "")

# count columns
print(f"Number of columns: {len(cleaned_df.columns)}")
# display_pretty(cleaned_df)

# Count NaNs vs non-NaNs
nans = cleaned_df.isna().sum()
non_nans = cleaned_df.count()
print(f"Number of NaNs: {nans.sum()}")
print(f"Number of non-NaNs: {non_nans.sum()}")

# NaNs per row with row key being the column 1
nans_per_row = cleaned_df.isna().sum(axis=1)
print(f"Number of NaNs per row: {nans_per_row}")

Number of columns: 167

Number of NaNs: 216

Number of non-NaNs: 2456

Number of NaNs per row: 0      0
1      2
2     47
3      5
4     47
5      2
6      2
7     33
8      0
9      5
10     5
11    47
12     0
13     5
14     2
15    14
dtype: int64